# Compiling different sources of sports data

In [36]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import datetime
import h3

#from fp.fp import FreeProxy

## Football: Chicago Bears

In [37]:
# URL of the webpage containing the NFL 2018 season data
response_football = requests.get("https://www.pro-football-reference.com/teams/chi/2018.htm")

soup = BeautifulSoup(response_football.content, "html.parser")
events = soup.find("table",id="games").tbody.findAll("tr")

row_constructor = {"Game_Start": [], "Duration": [], "Stadium": []}

for event in events:
    try:
        date = event.find("td",{"data-stat":"game_date"}).text + " 2018 "
        time = event.find("td",{"data-stat":"game_time"}).text.replace(" ET", "")
    
        datetime_obj = datetime.datetime.strptime(f"{date} {time}", "%B %d %Y %I:%M%p")
        row_constructor["Game_Start"].append(datetime_obj)
    except ValueError:
        print(f"Error parsing event. Row empty")
        
    #The information about the game location is stored on another endpoint for some reason. The code in the following would sent
    #a request for each game and retrieve this information. The Problem: The server starts to send Code 429 Too many requests
    #after some request. I implemented proxies to circumvent this. Unfortunately this is unstable and extremely slow. Thus, I
    #chose to collect the location and duration information manually. 
    '''
    try:
        boxscorelink = "https://www.pro-football-reference.com/" + event.find("td",{"data-stat":"boxscore_word"}).a["href"]
        print(boxscorelink)
        moreInfo = BeautifulSoup(requests.get(boxscorelink,proxies= {"https":FreeProxy(https=True).get()},verify=False).content,"html.parser")
        stadium = moreInfo.find("div",class_="scorebox_meta").findAll("div")[2].text
        if(stadium == "Soldier Field"):
            row_constructor["Date"].append(event.find("td",{"data-stat":"game_date"}).text)
            row_constructor["Time"].append(event.find("td",{"data-stat":"gametime"}).text)
            row_constructor["Duration"].append(moreInfo.find("div",class_="scorebox_meta").findAll("div")[4].text)
            row_constructor["Stadium"].append(stadium)
    except TypeError:
        print("Error parsing. Row has no boxscore.")    
    '''
    
row_constructor["Duration"] = [datetime.timedelta(minutes=0),
                               datetime.timedelta(minutes=179),
                               datetime.timedelta(minutes=0),
                               datetime.timedelta(minutes=177),
                               datetime.timedelta(minutes=0),
                               datetime.timedelta(minutes=201),
                               datetime.timedelta(minutes=180),
                               datetime.timedelta(minutes=0),
                               datetime.timedelta(minutes=193),
                               datetime.timedelta(minutes=183),
                               datetime.timedelta(minutes=0),
                               datetime.timedelta(minutes=0),
                               datetime.timedelta(minutes=192),
                               datetime.timedelta(minutes=178),
                               datetime.timedelta(minutes=0),
                               datetime.timedelta(minutes=0),
                               datetime.timedelta(minutes=189)
                              ]
row_constructor["Stadium"] = [False,True,False,True,False,True,True,False,True,True,False,False,True,True,False,False,True]

# Berechnung der Endzeiten der Spiele
row_constructor["Game_End"] = [
    start + duration for start, duration in zip(row_constructor["Game_Start"], row_constructor["Duration"])
]

nfldf = pd.DataFrame.from_dict(row_constructor)
nfldf = nfldf[nfldf["Stadium"]]
nfldf.drop(columns=["Duration","Stadium"],inplace=True)
nfldf["Latitude"] = 41.862366
nfldf["Longitude"] = -87.617256
nfldf["Team"] = "Chicago Bears"
nfldf["Sport"] = "Football"
nfldf.head(5)

Error parsing event. Row empty
Error parsing event. Row empty


Game_Start            Game_End   Latitude  Longitude  \
1 2018-09-17 20:15:00 2018-09-17 23:14:00  41.862366 -87.617256   
3 2018-09-30 13:00:00 2018-09-30 15:57:00  41.862366 -87.617256   
5 2018-10-21 13:00:00 2018-10-21 16:21:00  41.862366 -87.617256   
6 2018-10-28 13:00:00 2018-10-28 16:00:00  41.862366 -87.617256   
8 2018-11-11 13:00:00 2018-11-11 16:13:00  41.862366 -87.617256   

            Team     Sport  
1  Chicago Bears  Football  
3  Chicago Bears  Football  
5  Chicago Bears  Football  
6  Chicago Bears  Football  
8  Chicago Bears  Football

## Baseball: Chicago Cubs & Chicago White Sox

In [38]:
#This takes a minute unfortunately. We have to send a seperate request each time we want to know the game duration. 
#I dont know why but there are no games found during the season prior to April 5th.

data = requests.get("http://statsapi.mlb.com/api/v1/schedule/games/?sportId=1&startDate=2018-03-29&endDate=2018-09-30").json()

list_cubs = []
list_whitesox = []

for datum in data["dates"]:
    for game in datum["games"]:
        name = game["teams"]["home"]["team"]["name"]
        if(name == "Chicago Cubs"):
            print("Found Cubs home game. Opponent: ", game["teams"]["away"]["team"]["name"])
            gamedata_cubs = requests.get("http://statsapi.mlb.com" + game["link"]).json()    
            temp1 = [game["gameDate"],gamedata_cubs["gameData"]["gameInfo"]["gameDurationMinutes"]]
            list_cubs.append(temp1)
        elif(name == "Chicago White Sox"):
            print("Found White Sox home game. Opponent: ", game["teams"]["away"]["team"]["name"])
            gamedata_whitesox = requests.get("http://statsapi.mlb.com" + game["link"]).json()
            temp2 = [game["gameDate"],gamedata_whitesox["gameData"]["gameInfo"]["gameDurationMinutes"]]
            list_whitesox.append(temp2)
            
Chicago_Cubs_temp = pd.DataFrame(list_cubs)
Chicago_White_Sox_temp = pd.DataFrame(list_whitesox) 

Found White Sox home game. Opponent:  Detroit Tigers
Found White Sox home game. Opponent:  Detroit Tigers
Found White Sox home game. Opponent:  Detroit Tigers
Found White Sox home game. Opponent:  Tampa Bay Rays
Found Cubs home game. Opponent:  Pittsburgh Pirates
Found White Sox home game. Opponent:  Tampa Bay Rays
Found Cubs home game. Opponent:  Pittsburgh Pirates
Found White Sox home game. Opponent:  Tampa Bay Rays
Found Cubs home game. Opponent:  Pittsburgh Pirates
Found Cubs home game. Opponent:  Pittsburgh Pirates
Found Cubs home game. Opponent:  Atlanta Braves
Found Cubs home game. Opponent:  Atlanta Braves
Found Cubs home game. Opponent:  Atlanta Braves
Found Cubs home game. Opponent:  St. Louis Cardinals
Found Cubs home game. Opponent:  St. Louis Cardinals
Found Cubs home game. Opponent:  St. Louis Cardinals
Found Cubs home game. Opponent:  St. Louis Cardinals
Found White Sox home game. Opponent:  Houston Astros
Found White Sox home game. Opponent:  Houston Astros
Found White 

Found Cubs home game. Opponent:  Cincinnati Reds
Found White Sox home game. Opponent:  Chicago Cubs
Found White Sox home game. Opponent:  Chicago Cubs
Found White Sox home game. Opponent:  Chicago Cubs
Found Cubs home game. Opponent:  Pittsburgh Pirates
Found White Sox home game. Opponent:  Cleveland Indians
Found Cubs home game. Opponent:  Pittsburgh Pirates
Found White Sox home game. Opponent:  Cleveland Indians
Found Cubs home game. Opponent:  Pittsburgh Pirates
Found White Sox home game. Opponent:  Cleveland Indians
Found Cubs home game. Opponent:  Pittsburgh Pirates
Found Cubs home game. Opponent:  St. Louis Cardinals
Found Cubs home game. Opponent:  St. Louis Cardinals
Found Cubs home game. Opponent:  St. Louis Cardinals


In [39]:
#Chicago has an offset of GMT-5. We don't have to mess around with daylight saving time, because the switch does not occur during the season.
Chicago_Cubs_df = pd.DataFrame()
Chicago_Cubs_df["Game_Start"] = pd.to_datetime(Chicago_Cubs_temp[0]) - pd.to_timedelta(5, unit="h")
Chicago_Cubs_df["Game_End"] = Chicago_Cubs_df["Game_Start"] + pd.to_timedelta(Chicago_Cubs_temp[1], unit="m")
Chicago_Cubs_df["Latitude"] = 41.948463
Chicago_Cubs_df["Longitude"] = -87.655800
Chicago_Cubs_df["Team"]="Chicago Cubs"
Chicago_Cubs_df["Sport"]="Baseball"

Chicago_White_Sox_df = pd.DataFrame()
Chicago_White_Sox_df["Game_Start"] = pd.to_datetime(Chicago_White_Sox_temp[0]) - pd.to_timedelta(5, unit="h")
Chicago_White_Sox_df["Game_End"] = Chicago_White_Sox_df["Game_Start"] + pd.to_timedelta(Chicago_White_Sox_temp[1], unit="m")
Chicago_White_Sox_df["Latitude"] = 41.830017
Chicago_White_Sox_df["Longitude"] = -87.634598
Chicago_White_Sox_df["Team"]="Chicago White Sox"
Chicago_White_Sox_df["Sport"]="Baseball"

In [40]:
Chicago_White_Sox_df.head(5)

Game_Start                  Game_End   Latitude  Longitude  \
0 2018-04-05 15:10:00+00:00 2018-04-05 19:13:00+00:00  41.830017 -87.634598   
1 2018-04-07 13:10:00+00:00 2018-04-07 16:49:00+00:00  41.830017 -87.634598   
2 2018-04-08 13:10:00+00:00 2018-04-08 15:48:00+00:00  41.830017 -87.634598   
3 2018-04-09 13:10:00+00:00 2018-04-09 16:57:00+00:00  41.830017 -87.634598   
4 2018-04-10 13:10:00+00:00 2018-04-10 16:54:00+00:00  41.830017 -87.634598   

                Team     Sport  
0  Chicago White Sox  Baseball  
1  Chicago White Sox  Baseball  
2  Chicago White Sox  Baseball  
3  Chicago White Sox  Baseball  
4  Chicago White Sox  Baseball

In [41]:
Chicago_Cubs_df.head(5)

Game_Start                  Game_End   Latitude  Longitude  \
0 2018-04-09 14:20:00+00:00 2018-04-09 17:25:00+00:00  41.948463   -87.6558   
1 2018-04-10 13:20:00+00:00 2018-04-10 16:25:00+00:00  41.948463   -87.6558   
2 2018-04-11 19:05:00+00:00 2018-04-11 22:04:00+00:00  41.948463   -87.6558   
3 2018-04-12 13:20:00+00:00 2018-04-12 16:17:00+00:00  41.948463   -87.6558   
4 2018-04-13 13:20:00+00:00 2018-04-13 16:29:00+00:00  41.948463   -87.6558   

           Team     Sport  
0  Chicago Cubs  Baseball  
1  Chicago Cubs  Baseball  
2  Chicago Cubs  Baseball  
3  Chicago Cubs  Baseball  
4  Chicago Cubs  Baseball

## Chicago Blackhawks

In [42]:
# URL of the API containing the NHL 2017/2018 season data for the Chicago Blackhawks

response_hockey = requests.get("https://api-web.nhle.com/v1/club-schedule-season/CHI/20172018")
data = response_hockey.json()

nhldf_temp = pd.DataFrame(data["games"])

In [43]:
# Create mask for homeTeam ID
mask_home_team = nhldf_temp["homeTeam"].apply(lambda x: x['id']) == 16

# Create mask for gameDate
mask_game_date = pd.to_datetime(nhldf_temp["gameDate"]) >= pd.to_datetime("2018-01-01")

mask_combined = mask_home_team & mask_game_date

# Filter dataframe using mask
nhldf = nhldf_temp[mask_combined].copy()

#Unfortunately the API does not offer information about the duration of the game. I will use 2 hours and 19 minutes as average value.
nhldf.loc[:, 'venueUTCOffset'] = nhldf['venueUTCOffset'].apply(lambda x: int(x[:3]) + int(x[4:6])/60)
nhldf.loc[:, 'Game_Start'] = pd.to_datetime(nhldf["startTimeUTC"]) + pd.to_timedelta(nhldf["venueUTCOffset"], unit="h")
nhldf["Game_End"] = nhldf["Game_Start"] + pd.Timedelta(hours=2, minutes=19) 
nhldf = nhldf[["Game_Start","Game_End"]] 
nhldf["Latitude"] = 41.880692
nhldf["Longitude"] = -87.674370
nhldf["Team"] = "Chicago Blackhawks"
nhldf["Sport"] = "Hockey"
nhldf.head(5)

Game_Start                  Game_End   Latitude  Longitude  \
45 2018-01-05 19:30:00+00:00 2018-01-05 21:49:00+00:00  41.880692  -87.67437   
46 2018-01-07 14:00:00+00:00 2018-01-07 16:19:00+00:00  41.880692  -87.67437   
48 2018-01-10 19:00:00+00:00 2018-01-10 21:19:00+00:00  41.880692  -87.67437   
49 2018-01-12 19:30:00+00:00 2018-01-12 21:49:00+00:00  41.880692  -87.67437   
50 2018-01-14 11:30:00+00:00 2018-01-14 13:49:00+00:00  41.880692  -87.67437   

                  Team   Sport  
45  Chicago Blackhawks  Hockey  
46  Chicago Blackhawks  Hockey  
48  Chicago Blackhawks  Hockey  
49  Chicago Blackhawks  Hockey  
50  Chicago Blackhawks  Hockey

## Chicago Fire FC

In [44]:
# URL of the Wikipedia page

response = requests.get("https://en.wikipedia.org/wiki/2018_Chicago_Fire_season")

soup = BeautifulSoup(response.content, "html.parser")
events = soup.find_all("table", class_="tmpl-football-box-collapsible")

row_constructor = {"Game_Start": [], "Stadium": []}

for event in events:
    
    tr = event.tbody.find_all("tr")
    upper = tr[0].find_all("td")
    lower = tr[1].find_all("td")
    
    date = upper[0].find("span").text 
    time = lower[0].text
    try:
        datetime_obj = datetime.datetime.strptime(f"{date} {time}", "%B %d, %Y %H:%M")
        row_constructor["Game_Start"].append(datetime_obj)
        row_constructor["Stadium"].append(lower[4].find("span").text)
    except ValueError:
        print(f"Error parsing time for event: {date}")

mlsdf = pd.DataFrame.from_dict(row_constructor)
mlsdf = mlsdf[mlsdf["Stadium"] == "Toyota Park"]
mlsdf.drop(columns=["Stadium"],inplace=True)
#No information about the duration of the game. I will use 2 hours as average value.
mlsdf["Game_End"] = mlsdf["Game_Start"] + pd.Timedelta(hours=2)

#Manually adding row that got an parsing error (The wikipedia table is not consistent in structure) and occured in Toyota Park
mlsdf = pd.concat([mlsdf, pd.DataFrame([{"Game_Start":datetime.datetime(2018, 10, 28, 15, 30),"Game_End": datetime.datetime(2018, 10, 28, 17, 30)}])], ignore_index=True)
mlsdf["Latitude"] = 41.862366
mlsdf["Longitude"] = -87.617256
msldf["Team"] = "Chicago Fire FC"
msldf["Sport"] = "Soccer"
mlsdf.head(5)

Error parsing time for event: February 17, 2018
Error parsing time for event: October 21, 2018
Error parsing time for event: October 28, 2018
Error parsing time for event: August 8, 2018


Game_Start            Game_End   Latitude  Longitude
0 2018-03-03 12:00:00 2018-03-03 14:00:00  41.862366 -87.617256
1 2018-03-10 17:00:00 2018-03-10 19:00:00  41.862366 -87.617256
2 2018-03-31 17:00:00 2018-03-31 19:00:00  41.862366 -87.617256
3 2018-04-07 19:30:00 2018-04-07 21:30:00  41.862366 -87.617256
4 2018-04-14 14:30:00 2018-04-14 16:30:00  41.862366 -87.617256

## Chicago Bulls

In [46]:
#https://www.basketball-reference.com/teams/CHI/2018_games.html
#https://www.basketball-reference.com/teams/CHI/2019_games.html

response_nba2018 = requests.get("https://www.basketball-reference.com/teams/CHI/2018_games.html")
response_nba2019 = requests.get("https://www.basketball-reference.com/teams/CHI/2019_games.html")
soup2018 = BeautifulSoup(response_nba2018.content, "html.parser")
soup2019 = BeautifulSoup(response_nba2019.content, "html.parser")
events2018 = soup2018.find("table",id="games").tbody.findAll("tr")
events2019 = soup2019.find("table",id="games").tbody.findAll("tr")
events = events2018[37:] + events2019[:37] #Only looking at 2018 games

row_constructor = {"Game_Start": [], "Stadium": []}

for event in events:
    if(str(event.get("class")) == "['thead']"):
        continue
    
    date = event.find("td",{"data-stat":"date_game"}).text
    time = event.find("td",{"data-stat":"game_start_time"}).text.replace("p","PM")
    
    datetime_obj = datetime.datetime.strptime(f"{date} {time}", "%a, %b %d, %Y %I:%M%p")
    row_constructor["Game_Start"].append(datetime_obj)
    
    
    
    #The information about the game location is stored on another endpoint for some reason. The code in the following would sent
    #a request for each game and retrieve this information. The Problem: The server starts to send Code 429 Too many requests
    #after some request. I implemented proxies to circumvent this. Unfortunately this is unstable and extremely slow. Thus, I
    #chose to collect the location information manually. 
    ''' 
    if(str(event.get("class")) == "['thead']"):
        continue
    boxscorelink = "https://www.basketball-reference.com" + event.find("td",{"data-stat":"box_score_text"}).a["href"]
    boxscoreRequest = requests.get(boxscorelink,proxies= {"https":FreeProxy(https=True).get()},verify=False)
    moreInfo = BeautifulSoup(boxscoreRequest.content,"html.parser")
    stadium = moreInfo.find("div",class_="scorebox_meta").findAll("div")[1].text
    if(stadium == "United Center, Chicago, Illinois"):
        date = event.find("td",{"data-stat":"date_game"}).text
        time = event.find("td",{"data-stat":"game_start_time"}).text.replace("p","PM")
    
        datetime_obj = datetime.datetime.strptime(f"{date} {time}", "%a, %b %d, %Y %I:%M%p")
        row_constructor["Game_Start"].append(datetime_obj)
        row_constructor["Stadium"].append(stadium)
    '''
#True if game was played in Chicago. New line every 20 games.
row_constructor["Stadium"] = [False,True,False,True,True,False,False,True,False,True,False,False,True,False,False,False,False,True,True,False,
                             True,True,False,False,True,True,True,False,True,True,False,False,False,True,True,False,True,True,False,False,
                             True,False,True,True,True,False,False,False,True,True,False,False,False,False,True,True,True,True,False,False,
                             False,True,True,True,False,False,True,False,True,False,True,True,False,False,False,False,True,True,False,True,
                             False,True]
    
nbadf = pd.DataFrame.from_dict(row_constructor)
nbadf = nbadf[nbadf["Stadium"]]
nbadf.drop(columns=["Stadium"],inplace=True)
#We do not have information about game duration. I will use 2 hours and 30 minutes as average value. https://www.marca.com/en/basketball/nba/2022/03/10/622a267de2704ef25e8b4585.html
nbadf["Game_End"] = nbadf["Game_Start"] + pd.Timedelta(hours=2, minutes=30) 
nbadf["Latitude"] = 41.880692
nbadf["Longitude"] = -87.674370
nbadf["Team"] ="Chicago Bulls"
nbadf["Sport"]="Basketball"
nbadf.head(5)

Game_Start            Game_End   Latitude  Longitude  \
1 2018-01-03 20:00:00 2018-01-03 22:30:00  41.880692  -87.67437   
3 2018-01-06 19:00:00 2018-01-06 21:30:00  41.880692  -87.67437   
4 2018-01-08 20:00:00 2018-01-08 22:30:00  41.880692  -87.67437   
7 2018-01-15 15:30:00 2018-01-15 18:00:00  41.880692  -87.67437   
9 2018-01-20 17:00:00 2018-01-20 19:30:00  41.880692  -87.67437   

            Team       Sport  
1  Chicago Bulls  Basketball  
3  Chicago Bulls  Basketball  
4  Chicago Bulls  Basketball  
7  Chicago Bulls  Basketball  
9  Chicago Bulls  Basketball

# Geo data

In [47]:
def add_h3_location(row, resolution):
    return h3.geo_to_h3(row['Latitude'], row['Longitude'], resolution)

resolutions = [4,6,8]
sport_dfs = [nfldf,Chicago_White_Sox_df,Chicago_Cubs_df,nhldf,mlsdf,nbadf]

# H3-Location-Spalte hinzufügen
for df in sport_dfs:
    for res in resolutions:
        df[f"h3_res_{res}"] = df.apply(lambda row: add_h3_location(row, res), axis=1)

# Combine all dataframes and export to csv

In [48]:
sports_data = pd.concat(sport_dfs, ignore_index=True, sort=False)
sports_data.to_csv("sports_data.csv",index=False)

In [51]:
#These are all home games of these clubs in 2018 
sports_data.head()

Game_Start             Game_End   Latitude  Longitude  \
0  2018-09-17 20:15:00  2018-09-17 23:14:00  41.862366 -87.617256   
1  2018-09-30 13:00:00  2018-09-30 15:57:00  41.862366 -87.617256   
2  2018-10-21 13:00:00  2018-10-21 16:21:00  41.862366 -87.617256   
3  2018-10-28 13:00:00  2018-10-28 16:00:00  41.862366 -87.617256   
4  2018-11-11 13:00:00  2018-11-11 16:13:00  41.862366 -87.617256   

            Team     Sport         h3_res_4         h3_res_6         h3_res_8  
0  Chicago Bears  Football  842664dffffffff  862664c1fffffff  882664c1b1fffff  
1  Chicago Bears  Football  842664dffffffff  862664c1fffffff  882664c1b1fffff  
2  Chicago Bears  Football  842664dffffffff  862664c1fffffff  882664c1b1fffff  
3  Chicago Bears  Football  842664dffffffff  862664c1fffffff  882664c1b1fffff  
4  Chicago Bears  Football  842664dffffffff  862664c1fffffff  882664c1b1fffff